In [9]:
import pandas as pd
COLUMNS = ['target', 'ids', 'date', 'flag', 'user', 'text']
df = pd.read_csv('D:/Work/Programs/ML Programs/sentiment-analysis-project/data/data.csv', encoding='latin1', names=COLUMNS)

In [18]:
df = df[['target','text']]
df['target'] = df['target'].replace(4,1)
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [38]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

In [42]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [43]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text_tokens = text.split()
    cleaned_text = [word for word in text_tokens if word not in stop_words]
    cleaned_text = [lemmatizer.lemmatize(word) for word in cleaned_text]
    return " ".join(cleaned_text)

df['cleaned_text'] = df['text'].apply(clean_text)

In [46]:
from sklearn.model_selection import train_test_split
X = df['cleaned_text']
y = df['target']
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
xtrain_tfidf = vectorizer.fit_transform(xtrain)
xtest_tfidf = vectorizer.transform(xtest)

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
Logistic = LogisticRegression(max_iter=1000)
Logistic.fit(xtrain_tfidf, ytrain)
ypred = Logistic.predict(xtest_tfidf)
print(f"Accuracy: {accuracy_score(ytest, ypred):.4f}")
print("\nClassification Report:")
print(classification_report(ytest, ypred, target_names=['Negative', 'Positive']))

Accuracy: 0.7739

Classification Report:
              precision    recall  f1-score   support

    Negative       0.79      0.75      0.77    239361
    Positive       0.76      0.80      0.78    240639

    accuracy                           0.77    480000
   macro avg       0.77      0.77      0.77    480000
weighted avg       0.77      0.77      0.77    480000

